# Load Libraries

In [1]:
import os
import warnings
import logging
import sys
import pickle
import numpy as np
import pandas as pd
import geopandas as gpd
import dotenv
import pyet
import matplotlib.pyplot as plt

dotenv.load_dotenv()
logging.basicConfig(level=logging.INFO)
warnings.filterwarnings("ignore")
pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', '{:.2f}'.format)
pd.set_option('display.max_colwidth', None)

# Load Data

In [2]:
DATA_FOLDER_PATH = os.getenv('DATA_FOLDER_PATH')
MONTHLY_DATA_NAME = os.getenv('MONTHLY_DATA_NAME')

if not DATA_FOLDER_PATH or not MONTHLY_DATA_NAME:
    logging.error("Environment variables DATA_FOLDER_PATH or MONTHLY_DATA_NAME are not set.")
    sys.exit(1)

DATA_PATH = os.path.join(DATA_FOLDER_PATH, MONTHLY_DATA_NAME + '.parquet')
try:
    data = pd.read_parquet(path=DATA_PATH)
    logging.info(f"Data loaded successfully from {DATA_PATH}")
except Exception as e:
    logging.error(f"Error loading data from {DATA_PATH}: {e}")
    sys.exit(1)

INFO:root:Data loaded successfully from D:\\Pooya\\Storegs\\Dropbox\\IRIMO\\Export\Iran_Monthly_Data_1951_2025.parquet


In [3]:
data['date'] = pd.to_datetime(data[['year', 'month']].assign(day=15))

# Data Info

In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 693381 entries, 0 to 693380
Data columns (total 46 columns):
 #   Column             Non-Null Count   Dtype         
---  ------             --------------   -----         
 0   year               693381 non-null  int32         
 1   month              693381 non-null  int32         
 2   region_id          693381 non-null  object        
 3   region_name        693381 non-null  object        
 4   station_id         693381 non-null  object        
 5   station_name       693381 non-null  object        
 6   lat                693381 non-null  float64       
 7   lon                693381 non-null  float64       
 8   station_elevation  693381 non-null  float64       
 9   tmax               162557 non-null  float64       
 10  tmax_count         693381 non-null  int64         
 11  tmin               161808 non-null  float64       
 12  tmin_count         693381 non-null  int64         
 13  tm                 161595 non-null  float64 

# Calculate ETo

### Required Functions

In [5]:
def data_quality_control(
    df, 
    required_vars = ['tm']
):
    df.dropna(subset=required_vars, inplace=True, how='any')
    return df

In [6]:
def calculate_eto(group, name, func, required_vars):
    station = group['station_name'].iloc[0]
    print(f"Calculating ETo {name} For Station {station}")
    
    group = data_quality_control(
        df=group, 
        required_vars=required_vars
    )
    group = group.set_index('date').sort_index()
    
    if group.empty:
        return pd.Series(np.nan, index=group.index)
    
    missing = [var for var in required_vars if var not in group.columns]
    if missing:
        raise ValueError(f"Required variable(s) missing from the data: {missing}")

    kwargs = {}
    if "lat" in required_vars:
        kwargs["lat"] = pyet.utils.deg_to_rad(lat=group['lat'].iloc[0])
    if "station_elevation" in required_vars:
        kwargs["elevation"] = group['station_elevation'].iloc[0]
    if "sshn" in required_vars:
        kwargs["rs"] = pyet.calc_rad_sol_in(n=group["sshn"], lat=pyet.utils.deg_to_rad(lat=group['lat'].iloc[0]))
    if "ffm" in required_vars:
        kwargs["wind"] = group["ffm"]
    if "tm" in required_vars:
        kwargs["tmean"] = group['tm']
    if "tmax" in required_vars:
        kwargs["tmax"] = group['tmax']
    if "tmin" in required_vars:
        kwargs["tmin"] = group['tmin']
    if "um" in required_vars:
        kwargs["rh"] = group['um']
    if "umin" in required_vars:
        kwargs["rhmin"] = group['umin']
    if "umax" in required_vars:
        kwargs["rhmax"] = group['umax']

    eto = func(**kwargs)
    
    return eto

In [7]:
def calculate_eto_all_methods(group, dic_eto):
    results = {}
    for method, meta in dic_eto.items():
        df = group.copy()        
        res = calculate_eto(
            group=df,
            name=method,
            func=meta["func"],
            required_vars=meta["required_vars"]
        )
        results[method] = res
    return pd.DataFrame(results)

### ETo

In [8]:
dic_eto = {
    "Penman": {
        "func": pyet.penman,
        "required_vars": ["tm", "ffm", "sshn", "lat", "station_elevation", "tmax", "tmin", "um", "umin", "umax"]
    },
    "Penman-Monteith": {
        "func": pyet.pm,
        "required_vars": ["tm", "ffm", "sshn", "lat", "station_elevation", "tmax", "tmin", "um", "umin", "umax"]
    },
    "ASCE-PM": {
        "func": pyet.pm_asce,
        "required_vars": ["tm", "ffm", "sshn", "lat", "station_elevation", "tmax", "tmin", "um", "umin", "umax"]
    },
    "FAO-56": {
        "func": pyet.pm_fao56,
        "required_vars": ["tm", "ffm", "sshn", "lat", "station_elevation", "tmax", "tmin", "um", "umin", "umax"]
    },
    "Priestley-Taylor": {
        "func": pyet.priestley_taylor,
        "required_vars": ["tm", "sshn", "lat", "station_elevation", "tmax", "tmin", "um", "umin", "umax"]
    },
    "Kimberly-Penman": {
        "func": pyet.kimberly_penman,
        "required_vars": ["tm", "ffm", "sshn", "lat", "station_elevation", "tmax", "tmin", "um", "umin", "umax"]
    },
    "Thom-Oliver": {
        "func": pyet.thom_oliver,
        "required_vars": ["tm", "ffm", "sshn", "lat", "station_elevation", "tmax", "tmin", "um", "umin", "umax"]
    },
    "Blaney-Criddle": {
        "func": pyet.blaney_criddle,
        "required_vars": ["tm", "lat"]
    },
    "Hamon": {
        "func": pyet.hamon,
        "required_vars": ["tm", "lat"]
    },
    "Romanenko": {
        "func": pyet.romanenko,
        "required_vars": ["tm", "tmax", "tmin", "um", "umin", "umax"]
    },
    "Linacre": {
        "func": pyet.linacre,
        "required_vars": ["tm", "lat", "station_elevation", "tmax", "tmin"]
    },
    # "Haude": {
    #     "func": pyet.haude,
    #     "required_vars": ["tmax", "um"]
    # },
    "Turc": {
        "func": pyet.turc,
        "required_vars": ["tm", "sshn", "um"]
    },
    "Jensen-Haise": {
        "func": pyet.jensen_haise,
        "required_vars": ["tm", "sshn"]
    },
    "Mcguinness-Bordne": {
        "func": pyet.mcguinness_bordne,
        "required_vars": ["tm", "lat"]
    },
    "Hargreaves": {
        "func": pyet.hargreaves,
        "required_vars": ["tm", "tmax", "tmin", "lat"]
    },
    "FAO-24": {
        "func": pyet.fao_24,
        "required_vars": ["tm", "ffm", "sshn", "um", "station_elevation"]
    },
    "Abtew": {
        "func": pyet.abtew,
        "required_vars": ["tm", "sshn"]
    },
    "Makkink": {
        "func": pyet.makkink,
        "required_vars": ["tm", "sshn", "station_elevation"]
    },
    "Oudin": {
        "func": pyet.oudin,
        "required_vars": ["tm", "lat"]
    }
}

eto = data.groupby(
    ['region_id', 'region_name', 'station_id', 'station_name'],
    group_keys=True
).apply(calculate_eto_all_methods, dic_eto)

Calculating ETo Penman For Station Zidashte Taleghan
Calculating ETo Penman-Monteith For Station Zidashte Taleghan
Calculating ETo ASCE-PM For Station Zidashte Taleghan
Calculating ETo FAO-56 For Station Zidashte Taleghan
Calculating ETo Priestley-Taylor For Station Zidashte Taleghan
Calculating ETo Kimberly-Penman For Station Zidashte Taleghan
Calculating ETo Thom-Oliver For Station Zidashte Taleghan
Calculating ETo Blaney-Criddle For Station Zidashte Taleghan
Calculating ETo Hamon For Station Zidashte Taleghan
Calculating ETo Romanenko For Station Zidashte Taleghan
Calculating ETo Linacre For Station Zidashte Taleghan
Calculating ETo Turc For Station Zidashte Taleghan
Calculating ETo Jensen-Haise For Station Zidashte Taleghan
Calculating ETo Mcguinness-Bordne For Station Zidashte Taleghan
Calculating ETo Hargreaves For Station Zidashte Taleghan
Calculating ETo FAO-24 For Station Zidashte Taleghan
Calculating ETo Abtew For Station Zidashte Taleghan
Calculating ETo Makkink For Station 

In [9]:
if isinstance(eto.index, pd.MultiIndex):
    eto = eto.reset_index()
    data = pd.merge(data, eto, on=['region_id', 'region_name', 'station_id', 'station_name', 'date'], how='left')
else:
    data = data.join(eto)

data

,year,month,region_id,region_name,station_id,station_name,lat,lon,station_elevation,tmax,tmax_count,tmin,tmin_count,tm,tm_count,umax,umax_count,umin,umin_count,um,um_count,ffm,ffm_count,sshn,sshn_count,pm,pm_count,p0m,p0m_count,ewm,ewm_count,radglo24,radglo24_count,td_m,td_m_count,twet_m,twet_m_count,tsoil_m,tsoil_m_count,ewsm,ewsm_count,evt,evt_count,rrr24,rrr24_count,date,Penman,Penman-Monteith,ASCE-PM,FAO-56,Priestley-Taylor,Kimberly-Penman,Thom-Oliver,Blaney-Criddle,Hamon,Romanenko,Linacre,Turc,Jensen-Haise,Mcguinness-Bordne,Hargreaves,FAO-24,Abtew,Makkink,Oudin
0,1951,1,OIID,Airforce,40753,Dowshan Tappeh,35.70,51.48,1209.20,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,0.00,0,0.00,0,1951-01-15,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1951,2,OIID,Airforce,40753,Dowshan Tappeh,35.70,51.48,1209.20,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,0.00,0,0.00,0,1951-02-15,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1951,3,OIID,Airforce,40753,Dowshan Tappeh,35.70,51.48,1209.20,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,0.00,0,0.00,0,1951-03-15,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1951,4,OIID,Airforce,40753,Dowshan Tappeh,35.70,51.48,1209.20,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,0.00,0,0.00,0,1951-04-15,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1951,5,OIID,Airforce,40753,Dowshan Tappeh,35.70,51.48,1209.20,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,0.00,0,0.00,0,1951-05-15,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
693376,2025,5,OITZ,Zanjan,18699,Zarrenrood,35.75,48.48,1751.00,24.46,13,12.53,10,19.07,10,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,0.00,0,12.20,11,2025-05-15,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.57,4.42,NaN,5.07,NaN,NaN,5.76,4.77,NaN,NaN,NaN,3.92
693377,2025,6,OITZ,Zanjan,18699,Zarrenrood,35.75,48.48,1751.00,32.03,30,12.43,28,22.12,28,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,0.00,0,0.00,29,2025-06-15,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.20,5.75,NaN,6.37,NaN,NaN,6.78,6.92,NaN,NaN,NaN,4.61
693378,2025,7,OITZ,Zanjan,18699,Zarrenrood,35.75,48.48,1751.00,36.54,31,17.80,30,27.14,30,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,0.00,0,3.10,30,2025-07-15,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.82,7.63,NaN,8.37,NaN,NaN,7.91,7.49,NaN,NaN,NaN,5.38
693379,2025,8,OITZ,Zanjan,18699,Zarrenrood,35.75,48.48,1751.00,37.20,20,19.91,11,29.04,11,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,0.00,0,0.00,19,2025-08-15,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.70,7.60,NaN,9.22,NaN,NaN,7.68,6.88,NaN,NaN,NaN,5.23


# Convert Daily Mean ETo to Monthly Total ETo

In [10]:
days_in_month = data['date'].dt.daysinmonth

for col in list(dic_eto.keys()):
    data[col] = data[col] * days_in_month

data['date'] = pd.to_datetime(data[['year', 'month']].assign(day=1))
data = data.round(2)
data

,year,month,region_id,region_name,station_id,station_name,lat,lon,station_elevation,tmax,tmax_count,tmin,tmin_count,tm,tm_count,umax,umax_count,umin,umin_count,um,um_count,ffm,ffm_count,sshn,sshn_count,pm,pm_count,p0m,p0m_count,ewm,ewm_count,radglo24,radglo24_count,td_m,td_m_count,twet_m,twet_m_count,tsoil_m,tsoil_m_count,ewsm,ewsm_count,evt,evt_count,rrr24,rrr24_count,date,Penman,Penman-Monteith,ASCE-PM,FAO-56,Priestley-Taylor,Kimberly-Penman,Thom-Oliver,Blaney-Criddle,Hamon,Romanenko,Linacre,Turc,Jensen-Haise,Mcguinness-Bordne,Hargreaves,FAO-24,Abtew,Makkink,Oudin
0,1951,1,OIID,Airforce,40753,Dowshan Tappeh,35.70,51.48,1209.20,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,0.00,0,0.00,0,1951-01-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1951,2,OIID,Airforce,40753,Dowshan Tappeh,35.70,51.48,1209.20,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,0.00,0,0.00,0,1951-02-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1951,3,OIID,Airforce,40753,Dowshan Tappeh,35.70,51.48,1209.20,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,0.00,0,0.00,0,1951-03-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1951,4,OIID,Airforce,40753,Dowshan Tappeh,35.70,51.48,1209.20,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,0.00,0,0.00,0,1951-04-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1951,5,OIID,Airforce,40753,Dowshan Tappeh,35.70,51.48,1209.20,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,0.00,0,0.00,0,1951-05-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
693376,2025,5,OITZ,Zanjan,18699,Zarrenrood,35.75,48.48,1751.00,24.46,13,12.53,10,19.07,10,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,0.00,0,12.20,11,2025-05-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,110.63,136.90,NaN,157.27,NaN,NaN,178.47,147.74,NaN,NaN,NaN,121.41
693377,2025,6,OITZ,Zanjan,18699,Zarrenrood,35.75,48.48,1751.00,32.03,30,12.43,28,22.12,28,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,0.00,0,0.00,29,2025-06-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,125.91,172.37,NaN,191.24,NaN,NaN,203.47,207.46,NaN,NaN,NaN,138.41
693378,2025,7,OITZ,Zanjan,18699,Zarrenrood,35.75,48.48,1751.00,36.54,31,17.80,30,27.14,30,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,0.00,0,3.10,30,2025-07-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,149.49,236.46,NaN,259.57,NaN,NaN,245.22,232.24,NaN,NaN,NaN,166.81
693379,2025,8,OITZ,Zanjan,18699,Zarrenrood,35.75,48.48,1751.00,37.20,20,19.91,11,29.04,11,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,0.00,0,0.00,19,2025-08-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,145.58,235.50,NaN,285.95,NaN,NaN,238.18,213.22,NaN,NaN,NaN,162.03


# Export Data

In [11]:
data.to_parquet(DATA_FOLDER_PATH + '/Iran_Monthly_ETo_1951_2025.parquet')
logging.info("ETo data saved to pickle and parquet files.")

INFO:root:ETo data saved to pickle and parquet files.
